<a href="https://colab.research.google.com/github/mamikah/DeepLearning2022Late/blob/main/%E5%8F%8E%E9%9B%86%E3%81%97%E3%81%9F%E7%94%BB%E5%83%8F%E3%81%A7%E5%AD%A6%E7%BF%92%E3%81%A8%E5%88%86%E9%A1%9E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 収集した画像をColabにUPして学習させ、それをもとに分類させる

## ファイルのフォルダを準備する

In [ ]:
import os
# 分類対象の画像のフォルダを作成
os.makedirs('sample',exist_ok = True)
# 犬と判定されたときの置き場所
os.makedirs('output/dog/sample',exist_ok = True)